In [1]:
import torch
from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image
from torchvision import transforms
import json

In [2]:
# === LOAD MODEL AND PROCESSOR ===
model = AutoModelForImageClassification.from_pretrained("efficientnet-plant-disease").eval()
processor = AutoImageProcessor.from_pretrained("efficientnet-plant-disease")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

EfficientNetForImageClassification(
  (efficientnet): EfficientNetModel(
    (embeddings): EfficientNetEmbeddings(
      (padding): ZeroPad2d((0, 1, 0, 1))
      (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=valid, bias=False)
      (batchnorm): BatchNorm2d(32, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
      (activation): SiLU()
    )
    (encoder): EfficientNetEncoder(
      (blocks): ModuleList(
        (0): EfficientNetBlock(
          (depthwise_conv): EfficientNetDepthwiseLayer(
            (depthwise_conv_pad): ZeroPad2d((0, 1, 0, 1))
            (depthwise_conv): EfficientNetDepthwiseConv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same, groups=32, bias=False)
            (depthwise_norm): BatchNorm2d(32, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
            (depthwise_act): SiLU()
          )
          (squeeze_excite): EfficientNetSqueezeExciteLayer(
            (squeeze): AdaptiveAvgPool2d(output

In [4]:
# === LOAD LABEL MAP ===
with open("efficientnet-plant-disease\\label_map.json", "r") as f:
    label_map = json.load(f)

In [5]:
# === TRANSFORM ===
transform = transforms.Compose([
    transforms.Resize((260, 260)),
    transforms.ToTensor(),
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std)
])

In [6]:
# === PREDICT FUNCTION ===
def predict_image(img_path):
    image = Image.open(img_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # [1, 3, 260, 260]

    with torch.no_grad():
        outputs = model(image)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=1)
        pred_class_idx = torch.argmax(probs, dim=1).item()
        confidence = probs[0, pred_class_idx].item()

    class_name = label_map[str(pred_class_idx)]
    return class_name, confidence

In [7]:
img_path = "Screenshot 2025-08-05 095322.png"  # replace with your image
pred_class, conf = predict_image(img_path)
print(f"✅ Predicted: {pred_class} ({conf:.2%} confidence)")

✅ Predicted: Strawberry___Leaf_scorch (84.91% confidence)
